In [2]:
import os
import re
import cv2
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import logging
import csv
logging.basicConfig(filename='Log\\inception_v3.log',level=logging.DEBUG, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

In [4]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.layers.normalization import batch_normalization

# Train and Test Data Directory

In [3]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'

In [4]:
inception_model_dir = "PretrainedModel"

In [5]:
image_list = os.listdir(TRAIN_DIR)

# Extract Feature from Pretrined Inception

In [5]:
def create_graph():
  with tf.gfile.FastGFile(os.path.join(inception_model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [7]:
def extract_features(list_images, image_dir):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []
    create_graph()
    with tf.Session() as sess:
        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
        for ind, image in enumerate(list_images):
            try:
                path = os.path.join(image_dir,image)
                if (ind%100 == 0):
                    print('%d Processing %s...'  %(ind, image))
                if not gfile.Exists(path):
                    print('File does not exist %s', image)
                    tf.logging.fatal('File does not exist %s', image)

                image_data = gfile.FastGFile(path, 'rb').read()
                predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
                features[ind,:] = np.squeeze(predictions)
                labels.append(image.split('.')[0])
            except Exception as e:
                logging.error("Processing failed for %s", image)
                print (e)
                

    return features, labels

# Save the Features and Labels for Training Data

In [18]:
features,labels = extract_features(image_list, TRAIN_DIR)
print("Processing Completed")
pickle.dump(features, open('inception_features', 'wb'))
pickle.dump(labels, open('inception_labels', 'wb'))
print("Pickle file saved")

# Convert Class Label to One Hot Encoding

In [8]:
def one_hot_encode(word_label):
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

# Load the Features and Label of Training Data

In [9]:
features_train = pickle.load(open('inception_features', 'rb'))
labels_train = pickle.load(open('inception_labels', 'rb'))

In [10]:
one_hot_labels_train = []
for label in labels_train:
    one_hot = one_hot_encode(label)
    one_hot_labels_train.append(one_hot)

# Split the Training Data Set Into Train and Validation Set

In [12]:
X_train, X_validation, Y_train, y_validation = cross_validation.train_test_split(features_train, one_hot_labels_train, test_size=0.2, random_state=0)

# Fine Tuning Final Layer

In [6]:
BOTTLENECK_TENSOR_SIZE = 2048
class_count = 2


In [7]:
net = tflearn.input_data(shape=[None, BOTTLENECK_TENSOR_SIZE])
net = tflearn.fully_connected(net, 1024, activation = 'elu')
net = tflearn.fully_connected(net, 64, activation = 'elu')
net = tflearn.fully_connected(net, class_count, activation='softmax')
net = tflearn.regression(net, optimizer='momentum', loss='categorical_crossentropy', learning_rate=0.001)

model = tflearn.DNN(net, 
                    checkpoint_path='trained_model/fine_tune_inception/inception_v3.ckpt', 
                    tensorboard_verbose=3, max_checkpoints = 10,
                   tensorboard_dir = 'tensor_board')



In [15]:
model.fit(X_train, Y_train, n_epoch = 50, validation_set = (X_validation, y_validation), show_metric = True)

Training Step: 15649  | total loss: 0.00566 | time: 78.162s
| Momentum | epoch: 050 | loss: 0.00566 - acc: 0.9999 -- iter: 19968/20000
Training Step: 15650  | total loss: 0.00522 | time: 79.562s
| Momentum | epoch: 050 | loss: 0.00522 - acc: 0.9999 | val_loss: 0.01647 - val_acc: 0.9952 -- iter: 20000/20000
--


In [16]:
model.save('trained_model/tuned_inception_v3_model_momentum')

# Load the Fine-tuned Inception Model

In [11]:
model.load('trained_model/tuned_inception_v3_model')

# Extract Features for Test Data Set

In [20]:
test_features, test_labels = extract_features(test_imageList, TEST_DIR)
print("Processing Completed")
pickle.dump(test_features, open('test_inception_features', 'wb'))
pickle.dump(test_labels, open('test_inception_labels', 'wb'))
print("Pickle file saved")

# Load Features and Labels for Test Data

In [17]:
features_test = pickle.load(open('test_inception_features', 'rb'))
labels_test = pickle.load(open('test_inception_labels', 'rb'))

In [18]:
prediction = model.predict(features_test)


# Create the Submission File for Kaggle

In [19]:
l = list(zip(map(int,labels_test),prediction))

In [20]:
l  =sorted(l,key=lambda x: x[0])

In [21]:
l =list(map(lambda x:(x[0],x[1][1]),l))


In [22]:
total_dataset = len(labels_test)
output = []
outputFile = open("classification_report.csv", "w", newline = '')
fieldNames = ["id", "label"]
outputFileWriter = csv.DictWriter(outputFile, fieldnames = fieldNames, delimiter = ',')
outputFileWriter.writeheader()
for item in l:
    rowForWrite = {"id":item[0], "label":item[1]}
    outputFileWriter.writerow(rowForWrite)
    
outputFile.close()